# This file aims convert midi data to lists
- Read midi data to music21 streams
- Change music21 streams to part of chords and melody
- Change melody into rhythmic lists and corresponding list of notes
- Change rhythmic lists into rhythmic pattern strings (partitioned by measures, so do the list of melody notes and chords. in this scheme, we did not consider rhythm of chords. So, the size of chord list per measure is the same as that of melody)



In [0]:
import music21
from music21 import *

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#lib: from music21 import *
def type_is_note_or_chord(obj_type):
    if obj_type==note.Note or obj_type==chord.Chord:
        return True
    else:
        return False

def type_is_note_or_chord_or_rest(obj_type):
    if obj_type==note.Note or obj_type==chord.Chord or obj_type==note.Rest:
        return True
    else:
        return False

In [0]:
DURATION_EPS = 0.000001 #节奏的最小时值,不能设得过大，否则无法区分拍子；也不能设得太小，否则受到计算精度的影响

#Input type: <music21.stream.Part>
#Previous operation: read midi, convert to stream, get main instrument
#lib: from music21 import *
#Output type: [<music21.chord.Chord/<music21.note.Rest>/<music21.meter.TimeSignature>],[float/<cfractions.Fraction>]
#Discription: converge a instrumental part into accessible formation of list
def rectify_stream_part(stream_part,DURATION_EPS = 0.000001):
    symbol_list=[]#用于记录事件符号。如果是音符或者和弦，一律用和弦音升序排列的和弦obj来记载；如果是空拍，就按空拍obj来记载；如果是拍子转换，就用拍号的obj来记载
    time_stamp_list=[]#用于记录每个事件符号的起始时间
    L=len(stream_part)
    current_chord = chord.Chord([])
    
    for i in range(L):
        if i>=len(stream_part):
            break
        event=stream_part[i]
        event_time=event.offset
        # assert boolean_expression, 'assert if boolean_expression==False'
        assert len(time_stamp_list)==len(symbol_list), 'Pitch list not same length as rhythm!'
        type_of_event=type(event)
        
        if type_of_event==meter.TimeSignature:
            symbol_list.append(event)
            time_stamp_list.append(event_time)
            
        elif type_is_note_or_chord(type_of_event):
            current_chord.add(event)
            if i<len(stream_part)-1:
                next_event = stream_part[i+1]
                next_event_time = next_event.offset
                if type_is_note_or_chord(type(next_event)):#如果下一个事件还是一个音符或者和弦
                    if next_event_time-event_time<=DURATION_EPS:#如果是同时发生
                        continue #那么不管，读下一个event
                    else:#如果下一个音符或和弦发生在下一时刻，那么就记载这个和弦
                        symbol_list.append(current_chord)
                        time_stamp_list.append(event_time)
                        current_chord = chord.Chord([])
                elif type(next_event)==note.Rest:
                    if next_event_time-event_time<=DURATION_EPS: #如果音符和休止符同时发生
                        stream_part.pop(i+1)
                        continue #那么把这个没有意义的休止符扔掉，读下一个event
                else: #如果下一个事件既不是音符又不是和弦也不是休止符，那么就记载这个和弦
                    symbol_list.append(current_chord)
                    time_stamp_list.append(event_time)
                    current_chord = chord.Chord([])
            else:#如果读到的最后一个音，那么就记载这个和弦
                symbol_list.append(current_chord)
                time_stamp_list.append(event_time)
                current_chord = chord.Chord([])
        elif type_of_event==note.Rest:
            if i<len(stream_part)-1:
                next_event = stream_part[i+1]
                next_event_time = next_event.offset
                if type_is_note_or_chord(type(next_event)):#如果下一个事件是一个音符或者和弦
                    if next_event_time-event_time<=DURATION_EPS:#如果是同时发生
                        continue #那么忽略掉此时这个休止符，读下一个event
                    else:#如果下一个音符或和弦发生在下一时刻，那么就记载这个休止符
                        symbol_list.append(event)
                        time_stamp_list.append(event_time)
                elif type(next_event)==note.Rest:
                    if next_event_time-event_time<=DURATION_EPS: #如果多个休止符同时发生
                        stream_part.pop(i+1)
                        add=2
                        next_event = stream_part[i+add]
                        next_event_time = next_event.offset
                        while(next_event_time-event_time<=DURATION_EPS):
                            stream_part.pop(i+add)
                            add=add+1
                        continue #那么把下一个没有意义的休止符扔掉（假设连续的休止符是bug情况），读下一个event
                else: #如果下一个事件既不是音符又不是和弦也不是休止符，那么就记载这个休止符
                    symbol_list.append(event)
                    time_stamp_list.append(event_time)
            else:#如果读到的最后一个音，那么就记载这个休止符
                symbol_list.append(event)
                time_stamp_list.append(event_time)
    
    return symbol_list,time_stamp_list

In [0]:
# input type: [float/<cfractions.Fraction>]*L (len>=1)
# output type: [float/<cfractions.Fraction>]*L (len>=1)
def time2duration(time_stamp_list,last=None):
    length=len(time_stamp_list)
    duration_list = [None]*length
    for i in range(length-1):
        duration_list[i]=time_stamp_list[i+1]-time_stamp_list[i]
    duration_list[length-1]=last
    return duration_list

In [0]:
# for explanation (this dictionary not used)
rhythm_symbol_dictionary={'dotted_whole_note':'N6.000',
                          'dotted_whole_rest':'R6.000',
                          'dotted_whole_hold':'H6.000',
                          'whole_note':'N4.000',
                          'whole_rest':'R4.000',
                          'whole_hold':'H4.000',
                          'dotted_half_note':'N3.000',
                          'dotted_half_rest':'R3.000',
                          'dotted_half_hold':'H3.000',
                          'half_note':'N2.000',
                          'half_rest':'R2.000',
                          'half_hold':'H2.000',
                          'dotted_quarter_note':'N1.500',
                          'dotted_quarter_rest':'R1.500',
                          'dotted_quarter_hold':'H1.500',
                          'quarter_note':'N1.000',
                          'quarter_rest':'R1.000',
                          'quarter_hold':'H1.000',
                          'tri_quarter_note':'N0.667',
                          'tri_quarter_rest':'R0.667',
                          'tri_quarter_hold':'H0.667',
                          'quarter_note':'N0.667',
                          'quarter_rest':'R0.667',
                          'quarter_hold':'H0.667'}#and so forth
#'Note with duration T': 'N'+'{:.3f}'.format(note_duration)

In [0]:
#Rhythm Partition examples:
# 2/4: count_per_measure=2, beat_duration=1.0
# 3/4: count_per_measure=3, beat_duration=1.0
# 4/4: count_per_measure=4, beat_duration=1.0
# 5/4: count_per_measure=5, beat_duration=1.0
# 6/4: count_per_measure=6, beat_duration=1.0
# 3/8: count_per_measure=3, beat_duration=0.5
# 6/8: count_per_measure=6, beat_duration=0.5
# so forth

In [0]:
#format of rhythm pattern string: 'NoteOrRest,...,NoteOrRest|N/M'
#e.g. a rhythm pattern of 4/4 meter: 'H1.500,R0.500,N1.000,N1.000|4/4'
#e.g. changing the meter to 3/4: '|3/4'

#Input type: output from rectify_stream_part(stream_part)
#Previous operation: symbol_list,time_stamp_list=rectify_stream_part(stream_part)
#lib: from music21 import *
#Output type:
#   rhythm_pattern_list: [string],
#   rhythm_pattern_durations: [float],
#   melody_pitch_list: [list of integers],
#   melody_duration_list: [list of floats],
#Discription: converge a rectified list of chords into rhythm pattern and notes
def get_melody(symbol_list,time_list,is_duration=False,DURATION_EPS = 0.000001):
    count_per_measure=0 #每个小节数几拍，表示拍号的分子
    element_duration=0 #每拍多少时值
    duration_per_measure=0 #count_per_measure*element_duration
    duration_buffer=0 #现在已经数了多少时值
    
    meter_str=''
    
    #将时间点列表转换为时间段列表（如果本来输入的就是时间段列表那不用管）
    if not is_duration:
        if type_is_note_or_chord_or_rest(type(symbol_list[-1])):
            last_duration = symbol_list[-1].duration.quarterLength
        else:
            last_duration = 0
        duration_list = time2duration(time_list,last_duration)
    else:
        duration_list = time_list

    rhythm_pattern_list = []
    this_pattern = ''
    this_pattern_type = ''#N,R,H
    measure_complete_flag = False #记录是否读完一个小节
    split_flag = False #是否分裂到了两个小节
    rhythm_pattern_durations = []

    melody_pitch_list = [] #list of list
    melody_list_temp = []#list, -1表示空拍，-2表示其他控制符
    melody_duration_list = []# list of list
    melody_duration_temp = []# list

    last_pitch = None #用于比对这次的音和上一个小节的音是否保持，如果保持的话，用'H'去标记，否则用'N'
    this_pitch = None
    
    assert len(time_list)==len(symbol_list), 'Pitch list not same length as rhythm!'
    L=len(symbol_list)
    
    index=0
    while(index<len(symbol_list)):
        symbol=symbol_list[index]
        type_of_symbol=type(symbol)
        if type_of_symbol==meter.TimeSignature:
            count_per_measure=symbol.numerator
            #beat_duration=symbol.beatDuration.quarterLength
            meter_str=symbol.ratioString
            divide_index=meter_str.find('/')
            element_duration=round(4/float(meter_str[divide_index+1:]),3)
            duration_per_measure = count_per_measure*element_duration
            
            print(meter_str,duration_per_measure)
            duration_buffer=0#计数器清零，重新计时
            
            this_pitch=-2

            measure_complete_flag=False
            rhythm_pattern_list.append('|'+meter_str)
            rhythm_pattern_durations.append(round(0,3))
            melody_pitch_list.append([this_pitch])
            melody_duration_list.append([round(0,3)])

            melody_list_temp = []
            melody_duration_temp = []
            
        elif type_is_note_or_chord_or_rest(type_of_symbol):
            this_duration = duration_list[index]
            #print('this_duration',this_duration)
            duration_buffer = duration_buffer+this_duration
            
            if type_of_symbol==chord.Chord:
                this_pitch=symbol.pitches[-1].midi
                if this_pitch==last_pitch and split_flag:
                    this_pattern_type = 'H'
                else:
                    this_pattern_type = 'N'
            elif type_of_symbol==note.Note:
                this_pitch=symbol.pitch.midi
                if this_pitch==last_pitch and split_flag:
                    this_pattern_type = 'H'
                else:
                    this_pattern_type = 'N'
            elif type_of_symbol==note.Rest:
                this_pitch=-1
                this_pattern_type = 'R'

            last_pitch = this_pitch

            if duration_buffer-duration_per_measure > DURATION_EPS:
            #如果加入了这个和弦或者休止符以后，时值长度大于小节总长了的话
                overflow_duration = duration_buffer-duration_per_measure #计算多出来的时值
                left_duration = this_duration-overflow_duration #计算小节内剩下的时值
                if type_is_note_or_chord(type_of_symbol):
                    split_note_left = chord.Chord(symbol)
                    split_note_left.duration = duration.Duration(left_duration)
                    split_note_right = chord.Chord(symbol)
                    split_note_right.duration = duration.Duration(overflow_duration)
                else:
                    split_note_left = note.Rest(symbol)
                    split_note_left.duration = duration.Duration(left_duration)
                    split_note_right = note.Rest(symbol)
                    split_note_right.duration = duration.Duration(overflow_duration)
                #然后将这个和弦或者休止符分裂
                symbol_list[index]=split_note_left
                symbol_list.insert(index+1,split_note_right)
                duration_list[index]=left_duration
                duration_list.insert(index+1,overflow_duration)
                symbol=symbol_list[index]#重新取此时的符号
                
                split_flag = True
                measure_complete_flag=True
                duration_str_to_add = this_pattern_type+'{:.3f}'.format(float(left_duration))
                
            elif duration_buffer-duration_per_measure < -DURATION_EPS:
            #如果没有到小节时值，就把这个和弦或者休止符加进来，然后记载这一次的音符音高（以便与下一次比对，如果一样的话，就用hold）
                #print('duration_per_measure=',duration_per_measure)
                #print('duration_buffer=',duration_buffer)
                split_flag = False
                measure_complete_flag=False
                duration_str_to_add = this_pattern_type+'{:.3f}'.format(float(this_duration))
            
            else:
            #如果恰好读到小节结束
                split_flag = False
                measure_complete_flag=True
                duration_str_to_add = this_pattern_type+'{:.3f}'.format(float(this_duration))
            
            if measure_complete_flag:
                this_pattern = this_pattern+duration_str_to_add+'|'+meter_str
                melody_list_temp.append(this_pitch)
                if split_flag:
                    melody_duration_temp.append(round(left_duration,3))
                else:
                    melody_duration_temp.append(round(this_duration,3))
                
                rhythm_pattern_list.append(this_pattern)
                rhythm_pattern_durations.append(round(duration_buffer,3)) #精确到三位小数
                melody_pitch_list.append(melody_list_temp)
                melody_duration_list.append(melody_duration_temp)

                this_pattern=''
                duration_buffer = 0
                melody_list_temp = []
                melody_duration_temp = []

            else:
                this_pattern = this_pattern+duration_str_to_add+','
                melody_list_temp.append(this_pitch)
                if split_flag:
                    melody_duration_temp.append(round(left_duration,3))
                else:
                    melody_duration_temp.append(round(this_duration,3))

        index = index+1 #该干的干完了之后，index往后推一位
        
    assert len(rhythm_pattern_list)==len(rhythm_pattern_durations), 'Rhythm Pattern list not same length!'
    return rhythm_pattern_list, rhythm_pattern_durations, melody_pitch_list, melody_duration_list

In [0]:
#Discription: align the chord sequence to every notes (along with dufferent measures)
#Input type:
#   melody_duration_list: [list of floats],
#   chord_symbol_list: [music21 objects],
#   chord_time_list: [float] (same length as chord_symbol_list),
#   
#Previous operation: get melody_duration_list through get_melody(symbol_list,time_list)
#
#Output type:
#   aligned_chord_list: [list of music21 objects](the chord alignment in the same piece with melody_duration_list)
#lib: from music21 import *

def get_melody_aligned_with_chord(melody_duration_list,chord_symbol_list,chord_time_list,DURATION_EPS = 0.000001):
    time_stamp = 0
    chord_time_pointer = 0
    length_chord = len(chord_time_list)
    this_chord = []
    #last_chord = []
    this_measure_chord = []
    aligned_chord_list = []
    end_flag = False
    for measure in melody_duration_list:
        for note_dur in measure:
            time_this_chord = chord_time_list[chord_time_pointer] #起始时间
            if chord_time_pointer<length_chord-1:
                time_next_chord = chord_time_list[chord_time_pointer+1]
            else:
                time_next_chord = time_this_chord+100000 #ending
                end_flag = True
            
            if time_stamp>=time_this_chord and time_stamp<time_next_chord:
                this_chord = chord_symbol_list[chord_time_pointer]
            elif time_stamp>=time_next_chord:
                while time_stamp>=time_next_chord:#如果出现一个旋律对应很多和弦
                    time_this_chord=time_next_chord
                    if chord_time_pointer<length_chord-1:
                        chord_time_pointer = chord_time_pointer+1
                        this_chord = chord_symbol_list[chord_time_pointer]
                        if chord_time_pointer<length_chord-1:
                            time_next_chord = chord_time_list[chord_time_pointer+1]
                        else:
                            time_next_chord = time_this_chord+100000 #ending
                            end_flag = True
                    else:
                        this_chord = chord_symbol_list[length_chord-1]
                        break
                    if time_stamp>=time_this_chord and time_stamp<time_next_chord:
                        break
            else:
                this_chord = note.Rest(note_dur)
            
            this_measure_chord.append(this_chord)
            time_stamp = time_stamp+note_dur
            last_chord = this_chord

        aligned_chord_list.append(this_measure_chord)
        this_measure_chord=[]

    return aligned_chord_list


In [0]:
import os

filepath = '/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/dataset_NDH'
midi_files = os.listdir(filepath)


In [0]:
#Discription: read midi files to such objects:
#Outputs:
#   rhythm_pattern_list_all: [pieces],where every piece:[list_of_strings],
#   rhythm_pattern_duration_all: [float_lists],where every float_list:[floats(time_per_measure)],
#   melody_pitch_list_all: [pieces],where every piece:[list_of_integers],
#   melody_duration_list_all: [pieces],where every piece:[list_of_floats],
#   aligned_chord_list_all: [pieces],where every piece:[list_of_music21_objs],
#   chord_symbol_list_all: [pieces],where every piece:[music21_objs],
#   max_len_piece: integer
def midi2lists(filepath, make_controller=True):
    midi_files = os.listdir(filepath)
    rhythm_pattern_list_all = []
    rhythm_pattern_duration_all = []
    melody_pitch_list_all = []
    melody_duration_list_all = []
    aligned_chord_list_all = []

    chord_symbol_list_all = []
    for file in midi_files:
        if '.mid' not in file:
            continue
        mf = midi.MidiFile()
        mf.open(filepath + r'/' + file)
        mf.read()
        stream_tmp = midi.translate.midiFileToStream(mf)
        mf.close()
        print('read file',file)

        if len(stream_tmp.parts)==1:
            print('stream_tmp.parts las length 1, PASS!')
            continue
        elif len(stream_tmp.parts)==2:
            stream1_melody = stream_tmp.parts[0]
            stream2_chord = stream_tmp.parts[1]
        else:
            print('stream_tmp.parts las length more than 2, PASS!')
            continue
            
        instru = instrument.partitionByInstrument(stream1_melody)
        if instru:
            stream_part_melody = instrument.partitionByInstrument(stream1_melody).parts[0]
            stream_part_chord = instrument.partitionByInstrument(stream2_chord).parts[0]
        else:
            stream_part_melody = stream1_melody
            stream_part_chord = stream2_chord

        melody_symbol_list,melody_time_stamp_list=rectify_stream_part(stream_part_melody)
        chord_symbol_list,chord_time_stamp_list=rectify_stream_part(stream_part_chord)

        melody_rhythm_pattern_list, melody_rhythm_pattern_durations, melody_pitch_list, melody_duration_list = get_melody(melody_symbol_list,melody_time_stamp_list)
        aligned_chord_list = get_melody_aligned_with_chord(melody_duration_list,chord_symbol_list,chord_time_stamp_list)

        rhythm_pattern_list_all.append(melody_rhythm_pattern_list)
        rhythm_pattern_duration_all.append(melody_rhythm_pattern_durations)
        melody_pitch_list_all.append(melody_pitch_list)
        melody_duration_list_all.append(melody_duration_list)
        aligned_chord_list_all.append(aligned_chord_list)
        chord_symbol_list_all.append(chord_symbol_list)

        # print(rhythm_pattern_list_all[0])
        # print(melody_pitch_list_all[0])
        # print(melody_duration_list_all[0])
        # print(aligned_chord_list_all[0])
        # assert False, 'Debugging'
        
        max_len_piece=0
        if make_controller and rhythm_pattern_list_all[0][0]!='<BOS>':
            rest_note = note.Rest('')
            rest_note.duration = duration.Duration(0)
            for id_sen in range(len(rhythm_pattern_list_all)):
                #添加曲子开始和结束的标记
                rhythm_pattern_list_all[id_sen].insert(0,'<BOS>')
                rhythm_pattern_duration_all[id_sen].insert(0,0)
                melody_pitch_list_all[id_sen].insert(0,[-3])
                melody_duration_list_all[id_sen].insert(0,[0])

                aligned_chord_list_all[id_sen].insert(0,[rest_note])
                chord_symbol_list_all[id_sen].insert(0,rest_note)

                rhythm_pattern_list_all[id_sen].append('<EOS>')
                rhythm_pattern_duration_all[id_sen].append(0)
                melody_pitch_list_all[id_sen].append([-3])
                melody_duration_list_all[id_sen].append([0])
                aligned_chord_list_all[id_sen].append([rest_note])
                chord_symbol_list_all[id_sen].append(rest_note)          

        for id_sen in range(len(rhythm_pattern_list_all)):
            if len(rhythm_pattern_list_all[id_sen])>max_len_piece:
                max_len_piece=len(rhythm_pattern_list_all[id_sen])
    return rhythm_pattern_list_all,rhythm_pattern_duration_all,melody_pitch_list_all,melody_duration_list_all,aligned_chord_list_all,chord_symbol_list_all,max_len_piece


In [0]:
load_processed_data = True
if load_processed_data:
    rhythm_pattern_list_all,rhythm_pattern_duration_all,melody_pitch_list_all,melody_duration_list_all,aligned_chord_list_all,chord_symbol_list_all,max_len_piece=midi2lists(filepath, make_controller=True)


read file ashover36.mid
6/8 3.0
read file hpps5.mid
4/4 4.0
read file ashover28.mid
2/4 2.0
read file ashover37.mid
3/4 3.0
read file ashover4.mid
2/2 4.0
read file hpps1.mid
4/4 4.0
read file ashover29.mid
6/8 3.0
read file ashover38.mid
2/4 2.0
read file ashover5.mid
4/4 4.0
read file hpps33.mid
4/4 4.0
read file hpps21.mid
4/4 4.0
read file hpps4.mid
4/4 4.0
read file ashover39.mid
3/4 3.0
read file ashover30.mid
6/8 3.0
read file ashover6.mid
4/4 4.0
read file hpps6.mid
4/4 4.0
read file hpps20.mid
4/4 4.0
read file hpps38.mid
4/4 4.0
read file hpps39.mid
4/4 4.0
read file ashover41.mid
4/4 4.0
read file hpps9.mid
4/4 4.0
read file hpps40.mid
4/4 4.0
read file hpps22.mid
4/4 4.0
read file ashover40.mid
2/2 4.0
read file hpps44.mid
4/4 4.0
read file jigs51.mid
6/8 3.0
read file hpps7.mid
4/4 4.0
read file jigs57.mid
6/8 3.0
read file jigs58.mid
6/8 3.0
read file jigs59.mid
6/8 3.0
read file hpps41.mid
4/4 4.0
read file hpps23.mid
4/4 4.0
read file hpps43.mid
4/4 4.0
read file ashove

In [0]:
for i in range(len(rhythm_pattern_list_all[0])):
    print(rhythm_pattern_list_all[0][i])
    print(melody_pitch_list_all[0][i])
    print(melody_duration_list_all[0][i])
    print(aligned_chord_list_all[0][i])
    print('------------------------------')

<BOS>
[-3]
[0]
[<music21.note.Rest rest>]
------------------------------
|6/8
[-2]
[0]
[<music21.note.Rest rest>]
------------------------------
R2.500,N0.500|6/8
[-1, 71]
[2.5, 0.5]
[<music21.note.Rest rest>, <music21.note.Rest rest>]
------------------------------
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8
[69, 71, 72, 72, 71, 69]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
[<music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>]
------------------------------
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8
[81, 69, 69, 69, 71, 72]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
[<music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>]
------------------------------
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8
[69, 71, 72, 72, 71, 69]
[0.5, 0.5,

In [0]:
import pickle

data_folder_name = '/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder'

def save_data_lists(data_folder_name,rhythm_pattern_list_all,rhythm_pattern_duration_all,melody_pitch_list_all,melody_duration_list_all,aligned_chord_list_all,chord_symbol_list_all):
    filename1 = data_folder_name+'/rhythm_pattern_list_all.data'
    filename2 = data_folder_name+'/rhythm_pattern_duration_all.data'
    filename3 = data_folder_name+'/melody_pitch_list_all.data'
    filename4 = data_folder_name+'/melody_duration_list_all.data'
    filename5 = data_folder_name+'/aligned_chord_list_all.data'
    filename6 = data_folder_name+'/chord_symbol_list_all.data'
    # 存储变量的文件的名字
    # 以二进制写模式打开目标文件
    f1 = open(filename1, 'wb')
    f2 = open(filename2, 'wb')
    f3 = open(filename3, 'wb')
    f4 = open(filename4, 'wb')
    f5 = open(filename5, 'wb')
    f6 = open(filename6, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(rhythm_pattern_list_all, f1)
    pickle.dump(rhythm_pattern_duration_all, f2)
    pickle.dump(melody_pitch_list_all, f3)
    pickle.dump(melody_duration_list_all, f4)
    pickle.dump(aligned_chord_list_all, f5)
    pickle.dump(chord_symbol_list_all, f6)
    # 关闭文件
    f1.close()
    f2.close()
    f3.close()
    f4.close()
    f5.close()
    f6.close()
    max_len_piece=0
    for id_sen in range(len(rhythm_pattern_list_all)):
        if len(rhythm_pattern_list_all[id_sen])>max_len_piece:
            max_len_piece=len(rhythm_pattern_list_all[id_sen])
    print('sentence_max_length is: ',max_len_piece)
    print('e.g. sentence[0]: ',rhythm_pattern_list_all[0])
    return max_len_piece

def load_data_lists(data_folder_name):
    filename1 = data_folder_name+'/rhythm_pattern_list_all.data'
    filename2 = data_folder_name+'/rhythm_pattern_duration_all.data'
    filename3 = data_folder_name+'/melody_pitch_list_all.data'
    filename4 = data_folder_name+'/melody_duration_list_all.data'
    filename5 = data_folder_name+'/aligned_chord_list_all.data'
    filename6 = data_folder_name+'/chord_symbol_list_all.data'
    # 以二进制读模式打开目标文件
    f1 = open(filename1, 'rb')
    f2 = open(filename2, 'rb')
    f3 = open(filename3, 'rb')
    f4 = open(filename4, 'rb')
    f5 = open(filename5, 'rb')
    f6 = open(filename6, 'rb')
    # 将文件中的变量加载到当前工作区
    rhythm_pattern_list_all = pickle.load(f1)
    rhythm_pattern_duration_all = pickle.load(f2)
    melody_pitch_list_all = pickle.load(f3)
    melody_duration_list_all = pickle.load(f4)
    aligned_chord_list_all = pickle.load(f5)
    chord_symbol_list_all = pickle.load(f6)
    f1.close()
    f2.close()
    f3.close()
    f4.close()
    f5.close()
    f6.close()
    max_len_piece=0
    for id_sen in range(len(rhythm_pattern_list_all)):
        if len(rhythm_pattern_list_all[id_sen])>max_len_piece:
            max_len_piece=len(rhythm_pattern_list_all[id_sen])
    return rhythm_pattern_list_all,rhythm_pattern_duration_all,melody_pitch_list_all,melody_duration_list_all,aligned_chord_list_all,chord_symbol_list_all,max_len_piece

In [0]:
save_data_lists(data_folder_name,rhythm_pattern_list_all,rhythm_pattern_duration_all,melody_pitch_list_all,melody_duration_list_all,aligned_chord_list_all,chord_symbol_list_all)

sentence_max_length is:  513
e.g. sentence[0]:  ['<BOS>', '|6/8', 'R2.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N1.500,N1.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N1.500,N1.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0

In [0]:
def make_dictionary(rhythm_pattern_list_all):
    vocab_int2word={0:'<PAD>',1:'<BOS>',2:'<EOS>',3:'<UNK>'}
    vocab_word2int= dict(zip(vocab_int2word.values(), vocab_int2word.keys()))
    word_num = 4
    for piece in rhythm_pattern_list_all:
        for word in piece:
            if not (word in vocab_int2word.values()):
                #print(word)
                vocab_int2word.update({word_num:word})
                word_num=word_num+1
    vocab_word2int = {value:key for key,value in vocab_int2word.items()}
    return vocab_int2word,vocab_word2int


In [0]:
vocab_int2word,vocab_word2int=make_dictionary(rhythm_pattern_list_all)
print('total items:',len(vocab_int2word))
count=0
for id in vocab_int2word:
    print(vocab_int2word[id])


total items: 448
<PAD>
<BOS>
<EOS>
<UNK>
|6/8
R2.500,N0.500|6/8
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8
N1.500,N1.500|6/8
|4/4
R3.000,N0.333,N0.333,N0.333|4/4
N1.000,N1.000,N0.500,N0.500,N0.500,N0.500|4/4
N1.000,N1.000,N1.000,N1.000|4/4
N0.500,N0.500,N0.500,N0.500,N1.000,N1.000|4/4
N0.500,N0.500,N0.500,N0.500,N1.000,N0.333,N0.333,N0.333|4/4
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|4/4
N1.000,N1.000,N1.000,N0.333,N0.333,N0.333|4/4
|2/4
R1.000,N0.500,N0.500|2/4
N0.750,N0.250,N0.500,N0.500|2/4
N1.000,N0.500,N0.500|2/4
N0.500,N1.000,N0.250,N0.250|2/4
N0.500,N0.500,N0.500,N0.500|2/4
N1.500,N0.250,N0.250|2/4
|3/4
N2.000,N1.000|3/4
N1.500,N0.500,N1.000|3/4
N3.000|3/4
N1.000,N1.000,N1.000|3/4
|2/2
R3.000,N1.000|2/2
N0.750,N0.250,N0.750,N0.250,N0.750,N0.250,N0.750,N0.250|2/2
N1.000,N1.000,N1.000,N1.000|2/2
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|2/2
N0.500,N0.500,N0.500,N0.500,N0.500,N0.500,N1.000|2/2
N0.500,N0.500,N0.500,N0.500,N2.000|2/2
N1.000,N1.000,N2.0

In [0]:
import json

#int2word_json = json.dumps(vocab_int2word)

def save_dictionary_as_json(data_folder_name,vocab_int2word,vocab_word2int):
    filename3 = data_folder_name+'/vocab_int2word.json'
    filename4 = data_folder_name+'/vocab_word2int.json'
    # 存储变量的文件的名字
    # 以二进制写模式打开目标文件
    with open(filename3, 'w') as f3:
    # 将变量存储到目标文件中区
        json.dump(vocab_int2word, f3)
    # 关闭文件
    f3.close()
    with open(filename4, 'w') as f4:
    # 将变量存储到目标文件中区
        json.dump(vocab_word2int, f4)
    # 关闭文件
    f4.close()

def load_dictionary_as_json(data_folder_name):
    filename3 = data_folder_name+'/vocab_int2word.json'
    filename4 = data_folder_name+'/vocab_word2int.json'
    # 以二进制读模式打开目标文件
    f3 = open(filename3, 'rb')
    f4 = open(filename4, 'rb')
    # 将文件中的变量加载到当前工作区
    vocab_int2word = json.load(f3)
    vocab_word2int = json.load(f4)
    f3.close()
    f4.close()
    return vocab_int2word,vocab_word2int

In [0]:
#Discription: the inverse process of get_melody() for rhythm, with byproduct of duration list
#Input type:
#   rhythm_string: string(format: rhythm pattern),
#   place_for_controller: bool(if True, then generate Rests with duration 0 for controller strings),
#Output type:
#   symbol_list: [music21 objects](corresponding to rhythm_string),
#   duration_list: [floats](corresponding to rhythm_string)
#lib: from music21 import *
DURATION_EPS_FOR_RECONSTR = 0.01
def rhythm_pattern_string_to_duration_list(rhythm_string,place_for_controller=True,DURATION_EPS_FOR_RECONSTR = 0.01):
    duration_list=[]
    symbol_list=[]
    if rhythm_string[0] == '|':
        symbol_list.append(meter.TimeSignature(rhythm_string[1:]))
        duration_list.append(0)
    elif rhythm_string[0] == '<' and '|' not in rhythm_string:
        if place_for_controller:
            duration_list.append(0)
            this_obj = note.Rest('')
            this_obj.duration = duration.Duration(0)
            symbol_list.append(this_obj)
        else:
            pass
    else:
        meter_index=rhythm_string.find('|')
        meter_str=rhythm_string[meter_index+1:]

        divide_index=meter_str.find('/')
        element_duration=round(4/float(meter_str[divide_index+1:]),3)
        count_per_measure=int(meter_str[0:divide_index])
        string_duration=count_per_measure*element_duration
        #print('string_duration=',string_duration)

        tmp_str=rhythm_string.replace('|'+meter_str,'')
        obj_list=tmp_str.split(',')
        duration_buff=0

        for obj_str in obj_list:
            obj_type = obj_str[0]
            if obj_type=='H' or obj_type=='N':
                this_obj = note.Note('C5')
                this_duration = float(obj_str[1:])
            elif obj_type=='R':
                this_obj = note.Rest('')
                this_duration = float(obj_str[1:])
            elif obj_type=='<':
                if place_for_controller:
                    this_obj = note.Rest('')
                    this_duration = 0
                else:
                    continue
            if abs(duration_buff+this_duration-string_duration)<=DURATION_EPS_FOR_RECONSTR:
                this_duration = string_duration-duration_buff
            duration_list.append(this_duration)
            duration_buff+=this_duration
            this_obj.duration = duration.Duration(this_duration)
            symbol_list.append(this_obj)

        if abs(duration_buff-string_duration)>DURATION_EPS_FOR_RECONSTR:
            print('This rhythm string is '+rhythm_string)
            print('string_duration=',string_duration)
            print('duration_buff=',duration_buff)
            assert abs(duration_buff-string_duration)<=DURATION_EPS_FOR_RECONSTR, 'Note duration not compatable with meter!'
    return symbol_list,duration_list

In [0]:
#Discription: the elementwise inverse process of get_melody() for pitches and chords(optional)
#Input type:
#   note_list: [integers](for pitches),
#   duration_list: [integers](corresponding to note_list),
#   chord_alignment_list (optional): [music21 objects](corresponding to note_list),
#   rhythm_string (optional): string(format: rhythm pattern)
#Output type:
#   note_symbol_list: [music21 objects](for pitches),
#   chord_symbol_list (optional if chord_alignment_list): [music21 objects](durations rectified)
#lib: from music21 import *
def note_list_to_symbol_list(note_list,duration_list,chord_alignment_list=None,rhythm_string=None,DURATION_EPS_FOR_RECONSTR = 0.01):
    length=len(note_list)

    note_symbol_list = []
    if chord_alignment_list:
        chord_symbol_list = []
        this_chord = None
        last_chord = None
        this_chord_duration = 0
        chord_cum_duration = 0
    
    this_note = None
    sum_duration = sum(duration_list)

    correctness = True
    if len(note_list)!= len(duration_list):
        correctness = False
    if chord_alignment_list:
        if len(note_list)!= len(chord_alignment_list) or len(duration_list) != len(chord_alignment_list):
            correctness = False
    assert correctness, 'input not same length!'

    if note_list[0] == -3:
        rest_note = note.Rest('')
        rest_note.duration = duration.Duration(0)
        note_symbol_list.append(rest_note)
        if chord_alignment_list:
            chord_symbol_list.append(rest_note)
    elif note_list[0] == -2:
        if rhythm_string:
            if rhythm_string[0] == '|' and abs(duration_list[0])<=DURATION_EPS_FOR_RECONSTR:
                note_symbol_list.append(meter.TimeSignature(rhythm_string[1:]))
                if chord_alignment_list:
                    chord_symbol_list.append(meter.TimeSignature(rhythm_string[1:]))
            else:
                assert False, 'rhythm string or duration not compatable!'
    else:
        if rhythm_string:
            meter_index=rhythm_string.find('|')
            meter_str=rhythm_string[meter_index+1:]

            divide_index=meter_str.find('/')
            element_duration=round(4/float(meter_str[divide_index+1:]),3)
            count_per_measure=int(meter_str[0:divide_index])
            string_duration=count_per_measure*element_duration

            tmp_str = rhythm_string.replace('|'+meter_str,'')
            str_obj_list = tmp_str.split(',')
            duration_buff = 0
            assert abs(sum_duration-string_duration)<=DURATION_EPS_FOR_RECONSTR, 'rhythm string meter not compatable!'
        for i in range(length):
            this_note = note_list[i]
            this_duration = float(duration_list[i])
            if chord_alignment_list:
                this_chord = chord_alignment_list[i]
            if rhythm_string:
                this_type = tmp_str[i][0]
            else:
                this_type = 'N'

            if this_note == -1 or (bool(rhythm_string) and this_type=='H'):
                rest_note = note.Rest('')
                rest_note.duration = duration.Duration(this_duration)
                note_symbol_list.append(rest_note)
            else:
                pitch_note = note.Note(this_note)
                pitch_note.duration = duration.Duration(this_duration)
                note_symbol_list.append(pitch_note)
            if chord_alignment_list:
                if this_chord != last_chord:
                    this_chord_duration=this_duration
                    temp_i = i
                    if temp_i>=length-1:
                        this_chord_duration = sum_duration-chord_cum_duration
                    while temp_i<length-1:
                        if chord_alignment_list[temp_i+1]==this_chord:
                            this_chord_duration += float(duration_list[temp_i+1])
                            temp_i += 1
                        else:
                            break
                    chord_cum_duration+=this_chord_duration
                    this_chord_symbol = this_chord
                    this_chord_symbol.duration = duration.Duration(this_chord_duration)
                    chord_symbol_list.append(this_chord_symbol)
                last_chord = this_chord
    if chord_alignment_list:
        return note_symbol_list,chord_symbol_list
    else:
        return note_symbol_list,None


In [0]:
melody_pitch_list_test = melody_pitch_list_all[0]
melody_duration_list_test = melody_duration_list_all[0]
aligned_chord_list_test = aligned_chord_list_all[0]
rhythm_pattern_str_test = rhythm_pattern_list_all[0]
length_test = len(melody_pitch_list_test)
#print(note.Rest('').show('text'))

for i in range(length_test-6,length_test):
    note_symbol_list_test, _ = \
    note_list_to_symbol_list(melody_pitch_list_test[i], \
                            melody_duration_list_test[i], \
                            None, \
                            rhythm_pattern_str_test[i])
    print('----------'+str(i)+'----------')
    for current_note in note_symbol_list_test:
        current_note.show('text')

print('==================================')

for i in range(length_test-6,length_test):
    note_symbol_list_test,chord_symbol_list_test = \
    note_list_to_symbol_list(melody_pitch_list_test[i], \
                            melody_duration_list_test[i], \
                            aligned_chord_list_test[i], \
                            rhythm_pattern_str_test[i])
    print('----------'+str(i)+'----------')
    for current_note in note_symbol_list_test:
        current_note.show('text')
    for current_chord in chord_symbol_list_test:
        current_chord.show('text')

----------29----------
<music21.note.Note C>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note C>
----------30----------
<music21.note.Note B->
<music21.note.Note F>
<music21.note.Note D>
<music21.note.Note D>
<music21.note.Note F>
<music21.note.Note B>
----------31----------
<music21.note.Note C>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note C>
----------32----------
<music21.note.Note C>
<music21.note.Note C>
<music21.note.Note C>
<music21.note.Note C>
<music21.note.Note D>
<music21.note.Note E>
----------33----------
<music21.note.Note C>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note C>
----------34----------
<music21.note.Rest rest>
----------29----------
<music21.note.Note C>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note C>
<music21.chord.Chord 

In [0]:
rhythm_pttern_for_reconstr = rhythm_pattern_list_all[0][1]
symbol_list_reconstr,this_duration_reconstr = rhythm_pattern_string_to_duration_list(rhythm_pttern_for_reconstr,place_for_controller=True)
print(rhythm_pttern_for_reconstr)
print(symbol_list_reconstr)
print(this_duration_reconstr)
symbol_list_reconstr,this_duration_reconstr = rhythm_pattern_string_to_duration_list(rhythm_pttern_for_reconstr,place_for_controller=False)
print(rhythm_pttern_for_reconstr)
print(symbol_list_reconstr)
print(this_duration_reconstr)

|6/8
[<music21.meter.TimeSignature 6/8>]
[0]
|6/8
[<music21.meter.TimeSignature 6/8>]
[0]


In [0]:
#Discription: the inverse process of get_melody() for rhythm, with byproduct of duration list
#Input type:
#   string_list: [strings](format: rhythm pattern),
#   place_for_controller: bool(if True, then generate Rests with duration 0 for controller strings),
#Output type:
#   symbol_stream_tmp: music21 stream(corresponding to string_list),
#   duration_list: [list of floats](corresponding to string_list)
#lib: from music21 import *
def translate_rhythm_string_list_into_stream(string_list,place_for_controller=True,DURATION_EPS_FOR_RECONSTR = 0.01):
    symbol_list=[]
    duration_list=[]
    for string in string_list:
        decoded,durations = rhythm_pattern_string_to_duration_list(string,place_for_controller,DURATION_EPS_FOR_RECONSTR)
        if place_for_controller:
            symbol_list = symbol_list+decoded
            duration_list.append(durations)
        elif decoded: #decoded!=[]
            symbol_list = symbol_list+decoded
            duration_list.append(durations)
        else:
            continue

    symbol_stream_tmp=music21.stream.Stream()
    for obj in symbol_list:
        symbol_stream_tmp.append(obj)
    return symbol_stream_tmp,duration_list

In [0]:
#Discription: the inverse process of get_melody() for pitches and chords(optional)
#Input type:
#   note_measure_list: [list of integers](for pitches),
#   note_durations_list: [list of floats](corresponding to note_measure_list),
#   chord_measure_list (optional): [list of music21 objects](corresponding to note_measure_list),
#   rhythm_list (optional): [string](format: rhythm pattern)(corresponding to note_measure_list)
#Output type:
#   note_stream_tmp: music21 stream(for pitches),
#   chord_stream_tmp (optional if chord_measure_list): music21 stream(for chords)
#lib: from music21 import *
def translate_note_list_into_stream(note_measure_list,note_durations_list,chord_measure_list=None,rhythm_list=None,DURATION_EPS_FOR_RECONSTR = 0.01):
    note_list=[]
    if chord_measure_list:
        chord_list=[]

    assert len(note_measure_list) == len(note_durations_list), 'Length not compatable!'
    if chord_measure_list:
        assert len(note_measure_list) == len(chord_measure_list), 'Length not compatable!'
    if rhythm_list:
        assert len(note_measure_list) == len(rhythm_list), 'Length not compatable!'
    
    length = len(note_measure_list)
    for i in range(length):
        note_measure = note_measure_list[i]
        duration_measure = note_durations_list[i]

        if chord_measure_list:
            chord_measure = chord_measure_list[i]
        else:
            chord_measure = None
        if rhythm_list:
            rhythm_str = rhythm_list[i]
        else:
            rhythm_str = None

        note_list_temp,chord_list_temp = \
        note_list_to_symbol_list(note_measure, \
                            duration_measure, \
                            chord_measure, \
                            rhythm_str, DURATION_EPS_FOR_RECONSTR)

        note_list = note_list+note_list_temp
        if chord_measure_list:
            chord_list = chord_list+chord_list_temp

    #print(chord_list)

    note_stream_tmp=music21.stream.Stream()
    for obj in note_list:
        note_stream_tmp.append(obj)

    if chord_measure_list:
        chord_stream_tmp=music21.stream.Stream()
        last_obj = chord.Chord()
        for obj in chord_list:
            if obj.id==last_obj.id:
                #dur_sum = obj.duration.quarterLength+last_obj.duration.quarterLength
                #dur_obj = duration.Duration(dur_sum)
                #obj.duration = dur_obj
                #chord_stream_tmp[-1]=obj
                duraion_log = obj.duration
                if type(obj)==chord.Chord:
                    obj = chord.Chord(obj)
                elif type(obj)==note.Note:
                    obj = note.Note(obj)
                elif type(obj)==rest.rest:
                    obj = note.Rest(obj)
                else:
                    duraion_log = duration.Duration(0)
                    obj = note.Rest(obj)
                obj.duration = duraion_log
                chord_stream_tmp.append(obj)
            else:
                chord_stream_tmp.append(obj)
            last_obj = obj
    else:
        chord_stream_tmp=None

    return note_stream_tmp,chord_stream_tmp

In [0]:
rhythm_pttern_for_reconstr = rhythm_pattern_list_all[0][0:4]
symbol_stream_reconstr,duration_list_reconstr = translate_rhythm_string_list_into_stream(rhythm_pttern_for_reconstr,place_for_controller=True)
print(rhythm_pttern_for_reconstr)
print(symbol_stream_reconstr.show('text'))
print(duration_list_reconstr)

print('--------------------------------')

rhythm_pttern_for_reconstr = rhythm_pattern_list_all[0][0:4]
symbol_stream_reconstr,duration_list_reconstr = translate_rhythm_string_list_into_stream(rhythm_pttern_for_reconstr,place_for_controller=False)
print(rhythm_pttern_for_reconstr)
print(symbol_stream_reconstr.show('text'))
print(duration_list_reconstr)

['<BOS>', '|6/8', 'R2.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8']
{0.0} <music21.note.Rest rest>
{0.0} <music21.meter.TimeSignature 6/8>
{0.0} <music21.note.Rest rest>
{2.5} <music21.note.Note C>
{3.0} <music21.note.Note C>
{3.5} <music21.note.Note C>
{4.0} <music21.note.Note C>
{4.5} <music21.note.Note C>
{5.0} <music21.note.Note C>
{5.5} <music21.note.Note C>
None
[[0], [0], [2.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]
--------------------------------
['<BOS>', '|6/8', 'R2.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8']
{0.0} <music21.meter.TimeSignature 6/8>
{0.0} <music21.note.Rest rest>
{2.5} <music21.note.Note C>
{3.0} <music21.note.Note C>
{3.5} <music21.note.Note C>
{4.0} <music21.note.Note C>
{4.5} <music21.note.Note C>
{5.0} <music21.note.Note C>
{5.5} <music21.note.Note C>
None
[[0], [2.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]


In [0]:
melody_pitch_list_test = melody_pitch_list_all[0]
melody_duration_list_test = melody_duration_list_all[0]
aligned_chord_list_test = aligned_chord_list_all[0]
rhythm_pattern_str_test = rhythm_pattern_list_all[0]

note_stream_test,chord_stream_test = \
translate_note_list_into_stream(melody_pitch_list_test,melody_duration_list_test,aligned_chord_list_test,rhythm_pattern_str_test)

print(note_stream_test.show('text'))
print(chord_stream_test.show('text'))

[<music21.note.Rest rest>, <music21.meter.TimeSignature 6/8>, <music21.note.Rest rest>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord D2 F#2 A2>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord E2 G#2 B2>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord D2 F#2 A2>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord C2 E2 G2>, <music21.chord.Chord C2 E2 G2>, <music21.chord.Chord C2 E2 G2>, <music21.chord.Chord B-2 D3 F3>, <music21.chord.Chord C2 E2 G2>, <music21.chord.Chord C2 E2 G2>, <music21.cho

In [0]:
def result_tuplelist_to_midi(result_list_of_tuples,midi_results_path,max_file_num=None):
    if not max_file_num:
        max_file_num = len(result_list_of_tuples)
    for index,piece in enumerate(result_list_of_tuples):
        pred=piece[0]
        target=piece[1]
        print('Writing midi file number',index)
        test_stream_result=translate_rhythm_string_list_into_stream(pred)
        test_stream_target=translate_rhythm_string_list_into_stream(target)
        test_stream_result.write('midi', fp=midi_results_path+'/the_test_stream_result'+str(index)+'.mid')
        test_stream_target.write('midi', fp=midi_results_path+'/the_test_stream_target'+str(index)+'.mid')
        if index>max_file_num:
            break

def write_rhythm_patterns_to_midi(rhythm_patterns_lists,midi_results_path='/',file_name='/midi_thythm',max_file_num=None):
    if not max_file_num:
        max_file_num = len(rhythm_patterns_lists)
    for index,piece in enumerate(rhythm_patterns_lists):
        print('Writing midi file number',index)
        test_stream_result=translate_rhythm_string_list_into_stream(piece)
        test_stream_result.write('midi', fp=midi_results_path+file_name+str(index)+'.mid')
        if index>max_file_num:
            break

def write_notes_to_midi(notes_lists,duration_lists,aligned_chord_lists=None,rhythm_pattern_lists=None,midi_results_path='/',file_name='/midi_thythm',max_file_num=None):
    if not max_file_num:
        max_file_num = len(notes_lists)
    for i in range(max_file_num):
        notes_list = notes_lists[i]
        duration_list = duration_lists[i]
        if aligned_chord_lists:
            aligned_chord_list = aligned_chord_lists[i]
        else:
            aligned_chord_list = None
        if rhythm_pattern_lists:
            rhythm_pattern_list = rhythm_pattern_lists[i]
        else:
            rhythm_pattern_list = None
        
        note_stream,chord_stream = \
        translate_note_list_into_stream(notes_list,duration_list,aligned_chord_list,rhythm_pattern_list)

        print('Writing midi file (pitch+chord), number ',i)

        midi_score = stream.Score(id='mainScore')
        midi_part0 = stream.Part(id='part0')
        midi_part0.append(note_stream)
        midi_score.insert(0,midi_part0)
        if aligned_chord_lists:
            midi_part1 = stream.Part(id='part1')
            midi_part1.append(chord_stream)
            midi_score.insert(0,midi_part1)

        midi_score.write('midi', fp=midi_results_path+file_name+str(i)+'.mid')


In [0]:
import os
import music21
from music21 import *
import pickle
import json

## codes to add
def rhythm_pattern_to_n_note(rhythm_string):
    splitted = rhythm_string.split('|')
    if len(splitted)<=1:
        return 0
    if '' in splitted:
        return 0
    splitted2 = rhythm_string.split(',')

    if rhythm_string[0] == '|':
        return 0
    elif rhythm_string[0] == '<' and '|' not in rhythm_string:
        return 0
    else:
        meter_index=rhythm_string.find('|')
        meter_str=rhythm_string[meter_index+1:]
        tmp_str=rhythm_string.replace('|'+meter_str,'')
        obj_list=tmp_str.split(',')
        n_notes = 0
        for obj in obj_list:
            if '<' in obj:
                continue
            else:
                n_notes += 1
        return n_notes

def pitch2octave(pitch_list):
    if(type(pitch_list)==list):
        list_buff = [-1 for _ in range(len(pitch_list))]
        for i,p in enumerate(pitch_list):
            if p>=0:
                list_buff[i]=max(p//12-1,0)
            else:
                list_buff[i]=p
    else:
        p=pitch_list
        list_buff = -1
        if p>=0:
            list_buff=max(p//12-1,0)
        else:
            list_buff=p
    return list_buff

def pitch2pitchclass(pitch_list):
    if(type(pitch_list)==list):
        list_buff = [-1 for _ in range(len(pitch_list))]
        for i,p in enumerate(pitch_list):
            if p>=0:
                list_buff[i]=p%12
            else:
                list_buff[i]=p
    else:
        p=pitch_list
        list_buff = -1
        if p>=0:
            list_buff=p%12
        else:
            list_buff=p
    return list_buff

def chord_obj_to_string(chord_obj_list):
    list_buff = ['None' for _ in range(len(chord_obj_list))]
    for i,ch in enumerate(chord_obj_list):
        if type(ch)==note.Rest:
            list_buff[i] = 'Rest'
        elif type(ch)==chord.Chord:
            list_buff[i] = ch.commonName
        else:
            list_buff[i] = 'None'
    return list_buff

def chord_obj_to_bass(chord_obj_list):
    list_buff = ['None' for _ in range(len(chord_obj_list))]
    for i,ch in enumerate(chord_obj_list):
        if type(ch)==note.Rest:
            list_buff[i] = -1
        elif type(ch)==chord.Chord:
            list_buff[i] = ch.bass().pitchClass
        else:
            list_buff[i] = -3
    return list_buff

def pitch2onehot(pitch_num,PITCH_DIM_INDEX=127,REST_DIM_INDEX=128,ERROR_DIM_INDEX=129):
    #pitch dims: 0-127
    #rest dim: 128
    #error dim: 129
    pitch_num = int(pitch_num)
    one_hot_pitch = [0 for _ in range(ERROR_DIM_INDEX+1)]
    if pitch_num>=0 and pitch_num<=PITCH_DIM_INDEX:
        one_hot_pitch[pitch_num] = 1
    elif pitch_num==-1:
        one_hot_pitch[REST_DIM_INDEX] = 1
    elif pitch_num<=-2:
        one_hot_pitch[ERROR_DIM_INDEX] = 1
    else:
        print('Invalid pitch input!')
    return one_hot_pitch

def octave2onehot(octave_num,OCTAVE_DIM_INDEX=9,REST_DIM_INDEX=10,ERROR_DIM_INDEX=11):
    #octave dims: 0-9
    #rest dim: 10
    #error dim: 11
    octave_num = int(octave_num)
    one_hot_octave = [0 for _ in range(ERROR_DIM_INDEX+1)]
    if octave_num>=0 and octave_num<=OCTAVE_DIM_INDEX:
        one_hot_octave[octave_num] = 1
    elif octave_num==-1:
        one_hot_octave[REST_DIM_INDEX] = 1
    elif octave_num<=-2:
        one_hot_octave[ERROR_DIM_INDEX] = 1
    else:
        print('Invalid octave input!')
    return one_hot_octave

def pitchclass2onehot(pitchclass,CLASS_DIM_INDEX=11,REST_DIM_INDEX=12,ERROR_DIM_INDEX=13):
    #pitchclass dims: 0-11
    #rest dim: 12
    #error dim: 13
    pitchclass = int(pitchclass)
    one_hot_pitchclass = [0 for _ in range(ERROR_DIM_INDEX+1)]
    if pitchclass>=0 and pitchclass<=CLASS_DIM_INDEX:
        one_hot_pitchclass[pitchclass] = 1
    elif pitchclass==-1:
        one_hot_pitchclass[REST_DIM_INDEX] = 1
    elif pitchclass<=-2:
        one_hot_pitchclass[ERROR_DIM_INDEX] = 1
    else:
        print('Invalid pitchclass input!')
    return one_hot_pitchclass

class Nottingham_dataloader():
    def __init__(self,data_folder_name):
        self.data_folder_name = data_folder_name

        self.rhythm_pattern_list_all = []
        self.rhythm_pattern_duration_all = []
        self.melody_pitch_list_all = [] #-3:controller, -2:meter signature, -1:rest
        self.melody_duration_list_all = []
        self.aligned_chord_list_all = []
        self.aligned_chord_type_all = []
        self.aligned_chord_bass_all = []
        self.aligned_chord_func_all = []

        self.max_len_piece = 0
        self.vocab_int2word={}
        self.vocab_word2int={}

    def midi2lists(self,filepath, make_controller=True):
        self.rhythm_pattern_list_all, \
        self.rhythm_pattern_duration_all, \
        self.melody_pitch_list_all, \
        self.melody_duration_list_all, \
        self.aligned_chord_list_all, \
        self.chord_symbol_list_all, \
        self.max_len_piece = midi2lists(filepath, make_controller=True)
        print('data read successfully from midi files')

    def get_chord_types(self):
        print('converting chord types')
        n_piece = len(self.aligned_chord_list_all)
        self.aligned_chord_type_all = [[[] for _ in range(len(self.aligned_chord_list_all[i]))] for i in range(n_piece)]
        for i,piece in enumerate(self.aligned_chord_list_all):
            print("\r processing piece number ",i,end="",flush=True)
            for j,measure in enumerate(piece):
                self.aligned_chord_type_all[i][j]=chord_obj_to_string(measure)
        print('successfully converted chord types')

    def get_chord_bass(self):
        print('converting chord bass')
        n_piece = len(self.aligned_chord_list_all)
        self.aligned_chord_bass_all = [[[] for _ in range(len(self.aligned_chord_list_all[i]))] for i in range(n_piece)]
        for i,piece in enumerate(self.aligned_chord_list_all):
            print("\r processing piece number ",i,end="",flush=True)
            for j,measure in enumerate(piece):
                self.aligned_chord_bass_all[i][j]=chord_obj_to_bass(measure)
        print('successfully converted chord bass')


    def save_data_lists(self):
        save_data_lists(self.data_folder_name, \
                        self.rhythm_pattern_list_all, \
                        self.rhythm_pattern_duration_all, \
                        self.melody_pitch_list_all, \
                        self.melody_duration_list_all, \
                        self.aligned_chord_list_all, \
                        self.chord_symbol_list_all)
        print('data saved successfully to '+self.data_folder_name)
        
    def load_data_lists(self):
        print('loading data from'+self.data_folder_name)
        self.rhythm_pattern_list_all, \
        self.rhythm_pattern_duration_all, \
        self.melody_pitch_list_all, \
        self.melody_duration_list_all, \
        self.aligned_chord_list_all, \
        self.chord_symbol_list_all, \
        self.max_len_piece = load_data_lists(self.data_folder_name)
        print('data loaded successfully from'+self.data_folder_name)

    def make_dictionary(self):
        self.vocab_int2word,self.vocab_word2int = make_dictionary(self.rhythm_pattern_list_all)
        print('dictionary generated successfully')

    def save_dictionary_as_json(self):
        save_dictionary_as_json(self.data_folder_name,self.vocab_int2word,self.vocab_word2int)
        print('dictionary saved successfully to '+self.data_folder_name)

    def load_dictionary_as_json(self):
        print('loading dictionary from'+self.data_folder_name)
        self.vocab_int2word,self.vocab_word2int = load_dictionary_as_json(self.data_folder_name)
        print('dictionary loaded successfully from'+self.data_folder_name)

    def translate_rhythm_string_list_into_stream(self,string_list,place_for_controller=True,DURATION_EPS_FOR_RECONSTR = 0.01):
        symbol_stream,duration_list = \
        translate_rhythm_string_list_into_stream(string_list,place_for_controller=True,DURATION_EPS_FOR_RECONSTR = 0.01)
        return symbol_stream,duration_list
        
    def translate_note_list_into_stream(self,note_measure_list,note_durations_list,chord_measure_list=None,rhythm_list=None,DURATION_EPS_FOR_RECONSTR = 0.01):
        note_stream,chord_stream = \
        translate_note_list_into_stream(note_measure_list,note_durations_list,chord_measure_list=None,rhythm_list=None1)
        return note_stream,chord_stream

    def result_tuplelist_to_midi(self,result_list_of_tuples,midi_results_path='/',max_file_num=None):
        result_tuplelist_to_midi(result_list_of_tuples,midi_results_path,max_file_num)
        print('rhythm list tuple successfully saved to midi at'+midi_results_path)

    def write_rhythm_patterns_to_midi(self,rhythm_patterns_lists,midi_results_path='/',file_name='/midi_thythm',max_file_num=None):
        write_rhythm_patterns_to_midi(self,rhythm_patterns_lists,midi_results_path,file_name,max_file_num)
        print('rhythm list successfully saved to midi at'+midi_results_path)

    def write_notes_to_midi(self,notes_lists,duration_lists,aligned_chord_lists=None,rhythm_pattern_lists=None,midi_results_path='/',file_name='/midi_piece',max_file_num=None):
        write_notes_to_midi(notes_lists,duration_lists,aligned_chord_lists,rhythm_pattern_lists,midi_results_path,file_name,max_file_num)
        print('Successfully saved to midi at'+midi_results_path)


In [0]:
data_folder_name = '/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder'
loader_for_test = Nottingham_dataloader(data_folder_name)
filepath = '/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/dataset_NDH'
#loader_for_test.midi2lists(filepath, make_controller=True)
loader_for_test.load_data_lists()

loading data from/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder
data loaded successfully from/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder


In [0]:
loader_for_test.save_data_lists()

sentence_max_length is:  513
e.g. sentence[0]:  ['<BOS>', '|6/8', 'R2.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N1.500,N1.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N1.500,N1.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0.500|6/8', 'N0.500,N0.500,N0.500,N0.500,N0.500,N0

In [0]:
loader_for_test.make_dictionary()

dictionary generated successfully


In [0]:
loader_for_test.save_dictionary_as_json()

dictionary saved successfully to /content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder


In [0]:
loader_for_test.load_dictionary_as_json()

loading dictionary from/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder


In [0]:
notes_lists_test = [loader_for_test.melody_pitch_list_all[0]]
duration_lists_test = [loader_for_test.melody_duration_list_all[0]]
aligned_chord_lists_test = [loader_for_test.aligned_chord_list_all[0]]
rhythm_pattern_lists_test = [loader_for_test.rhythm_pattern_list_all[0]]

loader_for_test.write_notes_to_midi(notes_lists_test,duration_lists_test,aligned_chord_lists_test,rhythm_pattern_lists_test,midi_results_path=loader_for_test.data_folder_name,file_name='/midi_piece',max_file_num=None)

Writing midi file (pitch+chord), number  0
Successfully saved to midi at/content/drive/My Drive/Colab Notebooks/music_GAN_rhythm_seed/data_folder


In [0]:
loader_for_test.get_chord_types()

converting chord types
 processing piece number  1018successfully converted chord types


In [0]:
loader_for_test.get_chord_bass()

converting chord bass
 processing piece number  1018successfully converted chord bass


In [0]:
print(loader_for_test.aligned_chord_list_all[0])
print(loader_for_test.aligned_chord_type_all[0])
print(loader_for_test.aligned_chord_bass_all[0])

[[<music21.note.Rest rest>], [<music21.note.Rest rest>], [<music21.note.Rest rest>, <music21.note.Rest rest>], [<music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>], [<music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>], [<music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord D2 F#2 A2>, <music21.chord.Chord D2 F#2 A2>, <music21.chord.Chord D2 F#2 A2>], [<music21.chord.Chord G2 B2 D3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord G2 B2 D3>], [<music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chord.Chord A2 C3 E3>, <music21.chor

In [0]:
pitch2onehot(32)

In [0]:
octave2onehot(4)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [0]:
pitchclass2onehot(7)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]